Creates the database of the toolings by extracting the output file using RyR Generator target files

In [1]:
#% pip install openpyxl
import os, re, configparser
import pandas as pd
import numpy as np
from globals import glob
from utilities_database import prepare_data, prepare_database, consult_database, clear_databases, retrieve_data, rename_index, get_date, get_sigma, rename_limits_table, show_table
from utilities_analysis import mean_calculator, limits_generator, ini_generator, RyR, z_score_filter, reset_df
from utilities_plotting import plot_scatter, plot_capability
from utilities_widgets import widget_select_one
import _db_tools as db

Database *input.db* found in: c:\Users\lucio\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\input.db
Database *output.db* found in: c:\Users\lucio\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\output.db


In [2]:
##File filtering
extension = "xlsx"
file_list = os.listdir("../a1_input/")
filtered_list = [filename for filename in file_list if filename.endswith(extension)]
name = next((string for string in filtered_list if glob.tooling in string), None)
selected_button = widget_select_one(options=filtered_list, value=name, layout={'width': '600px'})
print("Files ready for storage:")
display(selected_button)

Files ready for storage:


Select(description='Files:', layout=Layout(width='600px'), options=(), rows=10, value=None)

In [3]:
##Database management
name = selected_button.value
print(f"File ready to add to the database: {name}")

File ready to add to the database: None


In [4]:
##Database generation
confirmation = input("You are going to modify the database. Are you sure you wrote the correct tooling in the globals file? (y/n): ").strip().lower()
if confirmation == 'y':
    try:
        file = selected_button.value 
        filepath = os.path.join(os.path.abspath("../a1_input/"), file)
    except:
        print("No selected file. Check the selection box.")
        raise SystemExit
    df = prepare_data(filepath)
    MEAS = prepare_data(filepath, filter="Measurements") #Load the output from RyR_Generator into a df
    LIMITS = prepare_data(filepath, filter="Limits") #Load the output from RyR_Generator into a df
    file, _ = os.path.splitext(file)
    file = re.sub(r'\W', '_', file)
    prepare_database("input.db", MEAS, file+"_Measurements") #Store a df for the measures inside the database of the project
    prepare_database("input.db", LIMITS, file+"_Limits") #Store a df for the limits inside the database of the project
    measurements_file = rename_limits_table("input.db", file+"_Measurements")
    limits_file = rename_limits_table("input.db", file+"_Limits")
    print("Files renamed for clarity.")
    table_names = consult_database("input.db", verbose=True)
else:
    print("Operation canceled.")

No selected file. Check the selection box.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [4]:
##Add tooling data. Ensure you updated the fixture tooling data in globals.glob
name = re.sub(r'\W', '_', glob.tooling + "_" + get_date()) + "_FixtureData"
confirmation = input("You are going to modify the database. Are you sure you wrote the correct tooling in the globals file? (y/n): ").strip().lower()
if confirmation == 'y':
    FIXTURE_DATA = pd.DataFrame(data=[glob.tooling, glob.lenses_per_nest, glob.nests_number,
                                glob.x_tolerance, glob.y_tolerance, glob.lo_tolerance,
                                glob.hi_tolerance], index=["Tooling name", "Lenses per nest",
                                "Number of nests", "X-axis tolerance", "Y-axis tolerance", "Lower tolerance",
                                "Higher tolerance"])
    FIXTURE_DATA.rename(columns={0:"Tooling data"}, inplace=True)
    prepare_database("input.db", FIXTURE_DATA, name, add_index=True) #Store a df for the fixture data inside the database of the project
    fixture_data = rename_limits_table("input.db", name)
    print("Files renamed for clarity.")
    #table_names = consult_database("input.db", verbose=True)
    FIXTURE_DATA = retrieve_data("input.db", fixture_data, index_col="index")
    print("    Review the fixture metadata:")
    display(FIXTURE_DATA)
else:
    print("Operation canceled.")

Dataframe stored as *VA_007_2024y_01m_25d_18h_45m_18s_FixtureData*
Files renamed for clarity.
Table *VA_007_FixtureData_2024y_01m_25d_18h_45m_18s* retrieved succesfully.
    Review the fixture metadata:


,Tooling data
Tooling name,VA_007
Lenses per nest,3
Number of nests,2
X-axis tolerance,0.0125
Y-axis tolerance,0.0165
Lower tolerance,0.02
Higher tolerance,0.03


In [7]:
###WARNING###
clear_databases(["input.db", "output.db"])

Database *input.db* cleared successfully.
Database *output.db* cleared successfully.
